In [1]:
import ee

# 初始化Earth Engine
ee.Authenticate()
ee.Initialize()

In [2]:
import pandas as pd
df= pd.read_csv(r'E:\NE_glb_cities.csv') #E:\NE_glb_cities.csv')
len(df)

1308302

In [3]:
#df=df.head(1)


In [6]:
import ee
import pandas as pd
import numpy as np
import concurrent.futures
from datetime import timedelta
import glob

# 定义要提取的变量
variables = [
    'u_component_of_wind_10m',  # 10m_u_component_of_wind
    'v_component_of_wind_10m',  # 10m_v_component_of_wind
    'dewpoint_temperature_2m',  # 2m_dewpoint_temperature
    'temperature_2m',  # 2m_temperature
    'soil_temperature_level_1',  # soil_temperature_level_1
    'surface_pressure',  # surface_pressure
    'leaf_area_index_high_vegetation',  # leaf_area_index_high_vegetation
    'leaf_area_index_low_vegetation',  # leaf_area_index_low_vegetation
    'volumetric_soil_water_layer_1',  # volumetric_soil_water_layer_1,
    'surface_net_solar_radiation_hourly', 
    'surface_net_thermal_radiation_hourly',

]

# 假设df是一个包含 'latitude', 'longitude', 'utc_time' 列的DataFrame
# 添加新列来存储提取的变量
for var in variables:
    df[var] = None



In [7]:
import ee
import pandas as pd
import numpy as np
import concurrent.futures
from datetime import timedelta
import glob

# 定义要提取的变量
variables = [
    'u_component_of_wind_10m',  # 10m_u_component_of_wind
    'v_component_of_wind_10m',  # 10m_v_component_of_wind
    'dewpoint_temperature_2m',  # 2m_dewpoint_temperature
    'temperature_2m',  # 2m_temperature
    'soil_temperature_level_1',  # soil_temperature_level_1
    'surface_pressure',  # surface_pressure
    'leaf_area_index_high_vegetation',  # leaf_area_index_high_vegetation
    'leaf_area_index_low_vegetation',  # leaf_area_index_low_vegetation
    'volumetric_soil_water_layer_1',  # volumetric_soil_water_layer_1,
    'surface_net_solar_radiation_hourly', 
    'surface_net_thermal_radiation_hourly',

]

# 假设df是一个包含 'latitude', 'longitude', 'utc_time' 列的DataFrame
# 添加新列来存储提取的变量
for var in variables:
    df[var] = None

# 将数据分成3个批次
n_splits = 10
dfs = np.array_split(df, n_splits)

def process_batch(batch_df, batch_index):
    # 在这里进行批次数据的处理
    for index, row in batch_df.iterrows():
        if index%10000==0:
            print("processing",index)
        latitude = row['latitude']
        longitude = row['longitude']
        utc_time = row['utc_time']
        
        # 将UTC时间转换为datetime格式
        utc_datetime = pd.to_datetime(utc_time)
        
        # 计算前后两小时的时间范围
        start_time = (utc_datetime - timedelta(hours=1)).strftime('%Y-%m-%dT%H:%M:%S')
        end_time = (utc_datetime + timedelta(hours=1)).strftime('%Y-%m-%dT%H:%M:%S')
        
        # 过滤数据集以匹配时间范围
        dataset = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY') \
                    .filterDate(start_time, end_time) \
                    .select(variables)
        
        # 指定感兴趣的地点
        point = ee.Geometry.Point([longitude, latitude])
        
        # 提取指定地点的最近时间点和最近经纬度点的数据
        data_at_point = dataset.first().reduceRegion(
            reducer=ee.Reducer.first(),
            geometry=point,
            scale=1000,
            bestEffort=True  # 尽量使用最佳的像素，避免空值
        ).getInfo()
        
        # 将提取的变量数据添加到DataFrame的相应行中
        for var in variables:
            batch_df.at[index, var] = data_at_point.get(var)
    
    # 保存批次数据到CSV文件
    batch_file_name = f'E:\\batch_result_{batch_index}.csv'
    batch_df.to_csv(batch_file_name, index=False)
    print(f"Batch {batch_index} saved as {batch_file_name}")

    return batch_file_name

D:\SOFTWARE\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [8]:
# 并行处理每个批次
with concurrent.futures.ThreadPoolExecutor() as executor:
    batch_files = list(executor.map(process_batch, dfs, range(n_splits)))

# 合并所有批次的CSV文件
csv_files = glob.glob("E:\\batch_result_*.csv")
df_merged = pd.concat((pd.read_csv(f) for f in csv_files), ignore_index=True)

# 保存最终合并结果
final_file_name = 'E:\\NE_glb_cities_era5_1019.csv'
df_merged.to_csv(final_file_name, index=False)
print(f"All batches have been merged and saved as {final_file_name}.")

processing 0
processing 1180000
processing 1050000
processing 920000
processing 790000
processing 660000
processing 530000
processing 400000
processing 270000
processing 140000
processing 10000
processing 1190000
processing 1060000
processing 930000
processing 800000
processing 670000
processing 540000
processing 410000
processing 280000
processing 150000
processing 20000
processing 1200000
processing 1070000
processing 940000
processing 810000
processing 680000
processing 550000
processing 420000
processing 290000
processing 160000
processing 30000
processing 1210000
processing 1080000
processing 950000
processing 820000
processing 690000
processing 560000
processing 430000
processing 300000
processing 170000
processing 40000
processing 1220000
processing 1090000
processing 960000
processing 830000
processing 700000
processing 570000
processing 440000
processing 310000
processing 180000
processing 50000
processing 1230000
processing 1100000
processing 970000
processing 840000
processi

In [19]:
print(len(df[df_merged['surface_net_solar_radiation_hourly'].isna()]))
print(len(df[df_merged['temperature_2m'].isna()]))
#print(len(df[df_merged['uv10m'].isna()]))
print(len(df[df_merged['leaf_area_index_high_vegetation'].isna()]))
print(len(df[df_merged['volumetric_soil_water_layer_1'].isna()]))
#len(df_merged['surface_net_solar_radiation_hourly'])
df_merged.columns

20958
21522
21522
21522


Index(['Unnamed: 0', 'latitude', 'longitude', 'utc_time', 'xco2', 'baseXCO2',
       'NE', 'local_time', 'year', 'month', 'hour', 'geometry', 'index_right',
       'COUNTRY', 'NAME_1', 'NAME_2', 'local_time_str',
       'u_component_of_wind_10m', 'v_component_of_wind_10m',
       'dewpoint_temperature_2m', 'temperature_2m', 'soil_temperature_level_1',
       'surface_pressure', 'leaf_area_index_high_vegetation',
       'leaf_area_index_low_vegetation',
       'volumetric_soil_water_layer_1surface_net_solar_radiation_hourly',
       'surface_net_thermal_radiation_hourly', 'volumetric_soil_water_layer_1',
       'surface_net_solar_radiation_hourly'],
      dtype='object')

In [21]:
df_merged['uv10m'] = np.sqrt(df_merged['u_component_of_wind_10m']**2 + df_merged['v_component_of_wind_10m']**2)
df_merged['LAI'] = df_merged['leaf_area_index_high_vegetation'] + df_merged['leaf_area_index_low_vegetation']
# 保存到 CSV 文件
df_merged.drop(columns = ['volumetric_soil_water_layer_1surface_net_solar_radiation_hourly'])
df_merged.to_csv(r'E:\\NE_glb_cities_era5_1020.csv')
df_merged.columns()

TypeError: 'Index' object is not callable